In [87]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import nltk

import re
import requests
import pandas as pd
import numpy as np
import json
import os
import dotenv
import sys
import lyricsgenius
import base64
from glob import glob
sys.tracebacklimit = 0 # turn off the error tracebacks

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from scipy.linalg import norm

In [88]:
OHCO = ['decade_id', 'song_num', 'stanza_num', 'line_num', 'token_num']

In [117]:
CORPUS = pd.read_csv('data/core_tables/CORPUS.csv', index_col= OHCO)
CORPUS

pos_tuple  pos  \
decade_id song_num stanza_num line_num token_num                            
0         0        0          0        0           ("Here's", 'NNP')  NNP   
                                       1               ('the', 'DT')   DT   
                                       2            ('thing,', 'NN')   NN   
                                       3               ('we', 'PRP')  PRP   
                                       4          ('started', 'VBD')  VBD   
...                                                              ...  ...   
5         107      9          2        1            ("y'all", 'PRP')  PRP   
                                       2             ('deep', 'VBP')  VBP   
                              3        0             ("Y'all", 'DT')   DT   
                                       1              ('are', 'VBP')  VBP   
                                       2            ('crazy,', 'JJ')   JJ   

                                                 token_str term_str pos_group  
decade_id song_num stanza_num line_num token_num                               
0         0        0          0        0            Here's    heres        NN  
                                       1               the      the        DT  
                                       2            thing,    thing        NN  
                                       3                we       we        PR  
                                       4           started  started        VB  
...                                                    ...      ...       ...  
5         107      9          2        1             y'all     yall        PR  
                                       2              deep     deep        VB  
                              3        0             Y'all     yall        DT  
                                       1               are      are        VB  
                                       2            crazy,    crazy        JJ  

[207176 rows x 5 columns]

In [90]:
VOCAB = pd.read_csv('data/core_tables/VOCAB.csv', index_col = 'term_str')
VOCAB

,n,n_chars,p,i,max_pos,max_pos_group,stem_porter,stop
term_str,,,,,,,,
1,2,1,0.000010,16.660497,CD,CD,1,0
10k,2,3,0.000010,16.660497,CD,CD,10k,0
1k,1,2,0.000005,17.660497,CD,CD,1k,0
1s,1,2,0.000005,17.660497,CD,CD,1s,0
2,3,1,0.000014,16.075535,CD,CD,2,0
...,...,...,...,...,...,...,...,...
è,12,1,0.000058,14.075535,NNP,NN,è,0
él,3,2,0.000014,16.075535,NNP,NN,él,0
еlla,1,4,0.000005,17.660497,IN,IN,еlla,0


In [91]:
def gather_docs(CORPUS, ohco_level, term_col='term_str'):
    OHCO = CORPUS.index.names
    CORPUS[term_col] = CORPUS[term_col].astype('str')
    DOC = CORPUS.groupby(OHCO[:ohco_level])[term_col].apply(lambda x:' '.join(x)).to_frame('doc_str')
    return DOC

In [92]:
DOC = gather_docs(CORPUS, 2)
DOC['n_tokens'] = DOC.doc_str.apply(lambda x: len(x.split()))

In [93]:
DOC = DOC.query("n_tokens > 1")

DOC

doc_str  \
decade_id song_num                                                      
0         0         heres the thing we started out friends it was ...   
          1         sweet disposition never too soon oh reckless a...   
          2         walking on a dream how can i explain talking t...   
          3         five four three two one ah you were a child cr...   
          4         if i told you things i did before told you how...   
...                                                               ...   
5         103       oohohohoh come on oh yeah well i tried to tell...   
          104       you and me we come from different worlds you l...   
          105       lifes like a road that you travel on when ther...   
          106       earn their lives again earn their li to pull u...   
          107       were going to dance were going to dance were g...   

                    n_tokens  
decade_id song_num            
0         0              279  
          1              231  
          2              252  
          3              220  
          4              307  
...                      ...  
5         103            517  
          104            299  
          105            390  
          106            453  
          107            382  

[651 rows x 2 columns]

# Create BOW

In [94]:
bags = dict(
    SENTS = OHCO[:4],
    STANZAS = OHCO[:3],
    SONGS = OHCO[:2],
    DECADES = OHCO[:1]
)

In [95]:
def create_bag_of_words(CORPUS, bag):
    BOW = CORPUS.groupby(bag+['term_str']).term_str.count().to_frame('n')
    return BOW

In [96]:
BOW = create_bag_of_words(CORPUS, bags['DECADES'])
BOW

n
decade_id term_str        
0         2g             1
          5              1
          5foot          1
          8th            1
          9              1
...                     ..
5         youve         44
          yow            1
          zombie         8
          zombieieie     2
          zombieieieie   2

[17678 rows x 1 columns]

# Compute TFIDF/DFIDF/DTM

In [97]:
def get_TFIDF(BOW, tf_method, idf_method ='standard'):
    DTCM = BOW.n.unstack(fill_value=0)

    DF = DTCM.astype('bool').sum() 
    N = len(DTCM)

    if tf_method == 'sum':
        TF = DTCM.T / DTCM.T.sum()

    elif tf_method == 'max':
        TF = DTCM.T / DTCM.T.max()
        
    elif tf_method == 'log':
        TF = np.log2(1 + DTCM.T)
        
    elif tf_method == 'raw':
        TF = DTCM.T
        
    elif tf_method == 'double_norm':
        TF = DTCM.T / DTCM.T.max()
        
    elif tf_method == 'binary':
        TF = DTCM.T.astype('bool').astype('int')
    
    TF = TF.T

    if idf_method == 'standard':
        IDF = np.log2(N / DF)

    elif idf_method == 'max':
        IDF = np.log2(DF.max() / DF) 

    elif idf_method == 'smooth':
        IDF = np.log2((1 + N) / (1 + DF)) + 1

    return [(TF * IDF), (DF*IDF), DTCM]

### TFIDF

In [98]:
TFIDF = get_TFIDF(BOW, 'max')[0]
TFIDF

term_str,1,10k,1k,1s,2,220kid,22nd,2g,2k,3,...,zone,zu,zum,zumban,zón,è,él,еlla,еsta,еyes
decade_id,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.001685,0.000000,0.000000,...,0.000652,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.001098,0.000000,0.00179,0.000000,0.000693,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001385,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000812,0.002647,0.00000,0.000000,0.000512,0.001324,0.001324,0.000000,0.003971,0.000000,...,0.000512,0.000000,0.000000,0.002647,0.001324,0.000000,0.003971,0.001324,0.001324,0.001324
3,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.00000,0.002219,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.002219,0.004438,0.000000,0.000000,0.026626,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.00000,0.000000,0.000670,0.000000,0.000000,0.000000,0.000000,0.003463,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [99]:
BOW['tfidf'] = TFIDF.stack()

In [100]:
BOW.sort_values('n', ascending = False).head(50)

n  tfidf
decade_id term_str             
2         i         1953    0.0
0         you       1534    0.0
2         you       1494    0.0
0         i         1493    0.0
5         i         1493    0.0
1         i         1444    0.0
          you       1444    0.0
0         the       1411    0.0
5         the       1209    0.0
1         the       1189    0.0
4         you       1165    0.0
2         the       1158    0.0
3         the       1002    0.0
5         you        983    0.0
4         i          970    0.0
          the        932    0.0
0         and        928    0.0
          to         924    0.0
1         me         870    0.0
3         you        811    0.0
2         me         794    0.0
          and        773    0.0
          a          737    0.0
1         and        732    0.0
5         and        723    0.0
0         a          707    0.0
1         to         704    0.0
0         me         695    0.0
5         to         685    0.0
1         my         667    0.0
3         a          659    0.0
          i          653    0.0
2         my         644    0.0
0         it         625    0.0
1         a          610    0.0
4         to         605    0.0
2         to         600    0.0
1         it         596    0.0
4         a          582    0.0
3         and        581    0.0
0         my         579    0.0
4         and        572    0.0
5         me         566    0.0
          a          560    0.0
0         im         527    0.0
5         it         515    0.0
2         im         513    0.0
4         it         511    0.0
2         in         511    0.0
4         me         504    0.0

### DFIDF (and add to VOCAB)

In [101]:
DFIDF = get_TFIDF(BOW, 'max')[1]
DFIDF = DFIDF.to_frame('dfidf')
DFIDF

,dfidf
term_str,
1,3.169925
10k,2.584963
1k,2.584963
1s,2.584963
2,3.000000
...,...
è,2.584963
él,2.584963
еlla,2.584963


In [102]:
# add DFIDF to VOCAB
VOCAB = VOCAB.merge(DFIDF, left_on='term_str', right_on='term_str', how = 'left')#.drop('dfidf_x', axis=1).rename({'dfidf_y':'dfidf'}, axis=1)
VOCAB

,n,n_chars,p,i,max_pos,max_pos_group,stem_porter,stop,dfidf
term_str,,,,,,,,,
1,2,1,0.000010,16.660497,CD,CD,1,0,3.169925
10k,2,3,0.000010,16.660497,CD,CD,10k,0,2.584963
1k,1,2,0.000005,17.660497,CD,CD,1k,0,2.584963
1s,1,2,0.000005,17.660497,CD,CD,1s,0,2.584963
2,3,1,0.000014,16.075535,CD,CD,2,0,3.000000
...,...,...,...,...,...,...,...,...,...
è,12,1,0.000058,14.075535,NNP,NN,è,0,2.584963
él,3,2,0.000014,16.075535,NNP,NN,él,0,2.584963
еlla,1,4,0.000005,17.660497,IN,IN,еlla,0,2.584963


## List the 20 most significant words in the corpus by dfidf

In [103]:
list(VOCAB.sort_values('dfidf', ascending=False).index[:20])

['1',
 'ridin',
 'extra',
 'ey',
 'roamin',
 'buzz',
 'buying',
 'timeless',
 'roadside',
 'rivers',
 'butter',
 'rises',
 'facts',
 'ting',
 'rips',
 'faded',
 'fags',
 'fail',
 'bursting',
 'rights']

### DTM

In [104]:
DTM = get_TFIDF(BOW, 'max')[2]
DTM

term_str,1,10k,1k,1s,2,220kid,22nd,2g,2k,3,...,zone,zu,zum,zumban,zón,è,él,еlla,еsta,еyes
decade_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,0,1,0,1,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
2,1,2,0,0,1,1,1,0,3,0,...,1,0,0,2,1,0,3,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,1,2,0,0,12,0,0,0,0
5,0,0,0,0,1,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0


## TFIDF_L2

In [105]:

TFIDF_reduced = BOW['tfidf'].unstack(fill_value=0)
pos_set = ['NN', 'VB']
VSHORT = VOCAB[VOCAB.max_pos_group.isin(['NN', 'VB', 'JJ']) & ~VOCAB.max_pos.isin(['NNP'])].sort_values('dfidf', ascending=False).head(5000)
TFIDF_reduced = TFIDF_reduced[VSHORT.index]

TFIDF_reduced

term_str,summat,tape,tamed,tan,pads,extra,ey,coat,internet,tap,...,irish,kufuffin,ladys,amar,lads,ladidah,laddie,ladadidada,ladadida,ladadadada
decade_id,,,,,,,,,,,,,,,,,,,,,
0,0.001033,0.001033,0.001033,0.000000,0.001033,0.000000,0.015498,0.001033,0.000000,0.002066,...,0.000000,0.001685,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
1,0.001098,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001098,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000812,0.000812,0.000812,0.021912,0.000000,0.001623,0.004869,...,0.002647,0.000000,0.000000,0.003971,0.002647,0.000000,0.00000,0.006618,0.001324,0.003971
3,0.000000,0.003164,0.000000,0.000000,0.000000,0.000000,0.000000,0.003164,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00516,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.001360,0.000000,0.000000,0.004081,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.002219,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.001062,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.003463,0.00000,0.000000,0.000000,0.000000


In [114]:
TFIDF_L2 = (TFIDF_reduced.T / norm(TFIDF_reduced, axis=1)).T

TFIDF_L2

term_str,summat,tape,tamed,tan,pads,extra,ey,coat,internet,tap,...,irish,kufuffin,ladys,amar,lads,ladidah,laddie,ladadidada,ladadida,ladadadada
decade_id,,,,,,,,,,,,,,,,,,,,,
0,0.006468,0.006468,0.006468,0.000000,0.006468,0.000000,0.097014,0.006468,0.000000,0.012935,...,0.00000,0.010548,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000
1,0.006871,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006871,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000
2,0.000000,0.000000,0.000000,0.004617,0.004617,0.004617,0.124660,0.000000,0.009234,0.027702,...,0.01506,0.000000,0.000000,0.02259,0.01506,0.000000,0.000000,0.03765,0.00753,0.02259
3,0.000000,0.010988,0.000000,0.000000,0.000000,0.000000,0.000000,0.010988,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.017921,0.00000,0.00000,0.00000
4,0.000000,0.000000,0.008398,0.000000,0.000000,0.025194,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.013697,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000
5,0.000000,0.000000,0.000000,0.004114,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.00000,0.013421,0.000000,0.00000,0.00000,0.00000


### Write Derived Tables to CSV

In [115]:
TFIDF.to_csv('data/derived_tables/TFIDF.csv')
DFIDF.to_csv('data/derived_tables/DFIDF.csv')
DTM.to_csv('data/derived_tables/DTM.csv')
DOC.to_csv('data/derived_tables/DOC.csv')
TFIDF_L2.to_csv('data/derived_tables/TFIDF_L2.csv')
BOW.to_csv('data/derived_tables/BOW.csv')

# Update VOCAB
VOCAB.to_csv('data/core_tables/VOCAB.csv')